O treinamento do modelo se deu sobre um subconjunto dos dados que foi anotado manualmente.  
O conjunto anotado pode ser encontrado em `sample.csv`.

A amostragem foi feita através do script `sample.py`.

In [1]:
import numpy as np
import pandas as pd
import re

import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
annotated_data = pd.read_csv('sample.csv')
annotated_data.head()

,TITLE,SMARTPHONE
0,Capa Samsung Galaxy S5 Mini Pc Couro Branco,0
1,Cabo Usb Retrátil 3 Adaptadores Preto - Muvit,0
2,Celular Samsung Galaxy A8 Plus 2018 Sm-a730f 6...,1
3,Lousa Magnética Grande Em Formato De Vaca Suelen,0
4,Aplique Madeira E Papel Placa Pinguim Com Colh...,0


# Extração de features

A extração de *features* foi feita através de expressões regulares:

Se o título de uma instância der match em uma regex em qualquer ponto, associa-se o valor 1 àquele atributo.  
Caso contrário, é atribuído 0.

In [3]:
patterns_re = [
    re.compile(r'smart', re.IGNORECASE), # contém "smart"
    re.compile(r'(?:ph|f)one', re.IGNORECASE), # contém "fone" ou "phone"
    re.compile(r'celular', re.IGNORECASE), # contém "celular"
    re.compile(r'bumper', re.IGNORECASE), # contém "bumper"
    re.compile(r'\b[a-z]\d\b', re.IGNORECASE), # contém, p.e., "G5", "S9", ...
    re.compile(r'\bcapa\b', re.IGNORECASE), # contém "capa"
    re.compile(r'\bpara\b', re.IGNORECASE), # contém "para" (p.e. "antena para celular")
]

# Usado para nomear as colunas do DataFrame
attr_names = ['smart', 'phone', 'celular', 'bumper', 'letra_num', 'capa', 'para']

In [4]:
# Transforma um título em uma lista de atributos
def get_attributes(title):
    title_attributes = []
    for pattern in patterns_re:
        if pattern.search(title) is None:
            title_attributes.append(0)
        else:
            title_attributes.append(1)
    return title_attributes

## Transformação dos dados

In [5]:
def get_row_attributes(dataframe):
    attributes = []
    for i in dataframe.index:
        row_attr = get_attributes(dataframe.loc[i].TITLE)
        attributes.append(row_attr)
    attr_df = pd.DataFrame(attributes)
    attr_df.columns = attr_names
    attr_df.insert(0, 'TITLE', dataframe.TITLE)
    return attr_df

In [6]:
annotated_attributes = get_row_attributes(annotated_data)
annotated_attributes['SMARTPHONE'] = annotated_data.SMARTPHONE

annotated_attributes.sample(10).sort_index()

,TITLE,smart,phone,celular,bumper,letra_num,capa,para,SMARTPHONE
88,Fonte Carregador Ibm Lenovo Thinkpad X60 X60s ...,0,0,0,0,0,0,0,0
131,Smartphone Samsung Galaxy Win 2 Duos TV G360BT...,1,1,0,0,0,0,0,1
174,Capa Lg L40 Pc Couro Preto,0,0,0,0,0,1,0,0
200,Película de vidro ultra temperado Asus Zenfone...,0,1,0,0,0,0,0,0
299,Fonte Carregador Sony Vaio Vgn-Fe880e/H Vgn-Fe...,0,0,0,0,0,0,0,0
318,Capa Samsung Galaxy Mini S5570 Tpu Transparente,0,0,0,0,0,1,0,0
343,"Whey Zero 4,4lbs - Black Skull",0,0,0,0,0,0,0,0
420,Película Protetora Para Motorola Moto X - Fosca,0,0,0,0,0,0,1,0
437,Adesivo Emborrachado Ursos Ad497 - Toke E Crie,0,0,0,0,0,0,0,0
498,Capa Transparente Para Microsoft Lumia 435,0,0,0,0,0,1,1,0


Aqui seria cabível uma análise de componentes principais ou análise de correspondência, mas por questões de tempo e simplicidade, deixei todas as colunas como estão.

# Avaliação de classificadores

In [7]:
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

from scipy.stats import ttest_ind

Foi usada a área sob a curva ROC como medida de avaliação.  
Nesse caso, pode ser interessante ajustar as taxas de falsos e verdadeiros positivos, 
logo a curva ROC é uma representação apropriada para a performance de um classificador.

Os classificadores testados foram o perceptron e a máquina de vetores de suporte com kernel polinomial de grau 3.  
Os resultados foram comparados com um teste T pareado para amostras independentes.

In [8]:
perceptron = Perceptron(max_iter=1e3)
svc = SVC(gamma='auto', kernel='poly', degree=3)

def eval_model(X, y):
    roc_auc_per = cross_val_score(
        perceptron,
        X,
        y,
        scoring='roc_auc',
        cv=10
    )
    print('Perceptron:')
    print(f'roc_auc: {np.mean(roc_auc_per):.2f} +- {np.std(roc_auc_per):.2f}\t'
         + f'min: {np.min(roc_auc_per):.2f}; max: {np.max(roc_auc_per):.2f}')
    
    roc_auc_svc = cross_val_score(
        svc,
        X,
        y,
        scoring='roc_auc',
        cv=10
    )
    print('SVC (poly-3):')
    print(f'roc_auc: {np.mean(roc_auc_svc):.2f} +- {np.std(roc_auc_svc):.2f}\t'
         + f'min: {np.min(roc_auc_svc):.2f}; max: {np.max(roc_auc_svc):.2f}')
    
    pvalue = ttest_ind(roc_auc_per, roc_auc_svc).pvalue
    print('=====')
    print(f'p-value: {pvalue:.3f} (5% de significância)')
    if pvalue < 0.05:
        print('Classificadores possuem performances diferentes')
    else:
        print('Não há evidência de diferença de performance')

In [9]:
# Todas as variáveis preditoras
eval_model(
    annotated_attributes.iloc[:, 1:-1].values,
    annotated_attributes.iloc[:, -1].values
)

Perceptron:
roc_auc: 0.97 +- 0.02	min: 0.93; max: 1.00
SVC (poly-3):
roc_auc: 0.97 +- 0.02	min: 0.94; max: 0.99
=====
p-value: 0.876 (5% de significância)
Não há evidência de diferença de performance


## Seleção do classificador

Em nenhum dos testes houve evidências para dizer que um dos classificadores teve performance melhor.  
Logo, optei pela simplicidade do perceptron em frente ao SVC. Entre os benefícios de optar pelo modelo simples, há a menor chance de overfitting e menor tempo de treino.

# Transformação do dataset

## Treino do classificador escolhido

In [10]:
classifier = Perceptron(max_iter=1e3)
X = annotated_attributes.iloc[:, 1:-1].values
y = annotated_attributes.iloc[:, -1].values
classifier.fit(X, y)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=1000.0, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=None,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
data = pd.read_csv('data_estag_ds.tsv', sep='\t')
data.head()

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita


In [12]:
data_attr = get_row_attributes(data)
data_attr.head()

,TITLE,smart,phone,celular,bumper,letra_num,capa,para
0,Acessório T - Jean Bag For Girls para DS Lite,0,0,0,0,0,0,1
1,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...,0,0,0,0,0,0,0
2,Carrinho de Bebê para Gêmeos Berço-Passeio - T...,0,0,0,0,0,0,1
3,Car Center - Calesita,0,0,0,0,0,0,0
4,Donka Trem com Som - Calesita,0,0,0,0,0,0,0


In [13]:
predictions = classifier.predict(data_attr.iloc[:, 1:].values)

In [14]:
answers = pd.DataFrame()
answers['ID'] = data.ID
answers['TITLE'] = data.TITLE
answers['SMARTPHONE'] = pd.Series(predictions)

In [15]:
answers.sample(10).sort_index()

,ID,TITLE,SMARTPHONE
15,3608270,Smartphone LG G2 Lite D295 Branco com Tela de ...,1
372,10413408,Cachorro Pula Pulga - Elka,0
391,10416928,Carregador Portátil Sony 5000mah Cp-V5 Branco,0
1039,10570266,Boneco Mickey Docinho Multibrink,0
1064,10570501,Papel Decoupage Flores Roxas Ld-797 - Litocart,0
1203,10588502,Capa Motorola Moto E PC Branco,0
1389,10615481,Sapatilha Metalizada Lilly's Closet Com Laço,0
1947,10664346,Fonte Carregador Ibm Lenovo Thinkpad Sl510 Sl5...,0
2145,708769702,Lançamento Smartphone Cat S61 Tela 5.2 64gb 4g...,1
2512,1081214087,Celular Lançamento Xiaomi Mi A2 64gb 4g Ram Gl...,1


In [16]:
answers.to_csv('output.tsv', sep='\t', index=False)